<a href="https://colab.research.google.com/github/gauravbansal98/Chatbot-using-tensorflow/blob/master/Chatbot_using_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib as plt
import random
import time
import re
from collections import Counter
import keras
from keras.preprocessing.sequence import pad_sequences
from tensorflow.python.layers import core as layers_core
from tensorflow.python.layers.core import Dense

import nltk
nltk.download()

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer

import os

os.chdir('drive/chatbot')

lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')
print(stop_words)

Using TensorFlow backend.


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /content/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /content/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package biocreative_ppi to /content/nltk_data...
       |   Package biocreative_ppi is already up-to-date!
       | Downloading package brown to /content/nltk_data...
       |   Package brown is already up-to-date!
       | Downloading package brown_tei to /content/nltk_data...
       |   Package brown_tei is already up-to-date!
       | Downloading package cess_cat to /content/nltk_data...
       |   Package cess_cat is already up-to-d

In [0]:
lines = open('movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conv_lines = open('movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

In [0]:
conv_lines[:5]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']"]

In [0]:
lines[:5]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go."]

In [0]:
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [0]:
id2line['L925']

"Let's go."

In [0]:
convs = [ ]
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    convs.append(_line.split(','))

In [0]:
convs[:10]

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366']]

In [0]:
questions = []
answers = []

for conv in convs:
    for i in range(len(conv)-1):
        questions.append(id2line[conv[i]])
        answers.append(id2line[conv[i+1]])

In [0]:
for i in range(5):
    print(questions[i])
    print(answers[i])
    print()

Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.
Well, I thought we'd start with pronunciation, if that's okay with you.

Well, I thought we'd start with pronunciation, if that's okay with you.
Not the hacking and gagging and spitting part.  Please.

Not the hacking and gagging and spitting part.  Please.
Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?

You're asking me out.  That's so cute. What's your name again?
Forget it.

No, no, it's my fault -- we didn't have a proper introduction ---
Cameron.



In [0]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

In [0]:
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
    
clean_answers = []    
for answer in answers:
    clean_answers.append(clean_text(answer))

In [0]:
for i in range(5):
    print(clean_questions[i])
    print(clean_answers[i])
    print()

can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again
well i thought we would start with pronunciation if that is okay with you

well i thought we would start with pronunciation if that is okay with you
not the hacking and gagging and spitting part  please

not the hacking and gagging and spitting part  please
okay then how about we try out some french cuisine  saturday  night

you are asking me out  that is so cute that is your name again
forget it

no no it is my fault  we did not have a proper introduction 
cameron



In [0]:
lengths = []
print(time.time())
for question in clean_questions:
    lengths.append(len(question.split()))
for answer in clean_answers:
    lengths.append(len(answer.split()))
print(time.time())

# Create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns=['counts'])
print(lengths[:5])

1527872746.411245
1527872746.8546896
   counts
0      22
1      14
2       9
3      14
4      13


In [0]:
lengths.describe()

,counts
count,443232.000000
mean,10.872094
std,12.215895
min,0.000000
25%,4.000000
50%,7.000000
75%,14.000000
max,555.000000


In [0]:
print(np.percentile(lengths, 25))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))
print(np.percentile(lengths, 99))

4.0
19.0
24.0
32.0
58.0


In [0]:
# Remove questions and answers that are shorter than 2 words and longer than 20 words.
min_line_length = 2
max_line_length = 20

# Filter out the questions that are too short/long
short_questions_temp = []
short_answers_temp = []

i = 0
for question in clean_questions:
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
        short_questions_temp.append(question)
        short_answers_temp.append(clean_answers[i])
    i += 1

# Filter out the answers that are too short/long
short_questions = []
short_answers = []

i = 0
for answer in short_answers_temp:
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(short_questions_temp[i])
    i += 1

In [0]:
print("# of questions:", len(short_questions))
print("# of answers:", len(short_answers))
print("% of data used: {}%".format(round(len(short_questions)/len(questions),4)*100))

# of questions: 138335
# of answers: 138335
% of data used: 62.419999999999995%


In [0]:
vocab = {}
for question in short_questions:
    for word in question.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1
            
for answer in short_answers:
    for word in answer.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

In [0]:
threshold = 10
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

In [0]:
print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)

Size of total vocab: 45618
Size of vocab we will use: 8092


In [0]:
questions_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        questions_vocab_to_int[word] = word_num
        word_num += 1
        
answers_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        answers_vocab_to_int[word] = word_num
        word_num += 1

In [0]:
codes = ['<PAD>','<EOS>','<UNK>','<GO>']

for code in codes:
    questions_vocab_to_int[code] = len(questions_vocab_to_int)+1 
    
for code in codes:
    answers_vocab_to_int[code] = len(answers_vocab_to_int)+1

In [0]:
questions_int_to_vocab = {v_i: v for v, v_i in questions_vocab_to_int.items()}
answers_int_to_vocab = {v_i: v for v, v_i in answers_vocab_to_int.items()}

In [0]:
print((questions_vocab_to_int))
print(len(questions_int_to_vocab))
print(len(answers_vocab_to_int))
print((answers_int_to_vocab))
questions_vocab_to_int['<GO>']

{'well': 0, 'i': 1, 'thought': 2, 'we': 3, 'would': 4, 'start': 5, 'with': 6, 'if': 7, 'that': 8, 'is': 9, 'okay': 10, 'you': 11, 'not': 12, 'the': 13, 'and': 14, 'spitting': 15, 'part': 16, 'please': 17, 'are': 18, 'asking': 19, 'me': 20, 'out': 21, 'so': 22, 'cute': 23, 'your': 24, 'name': 25, 'again': 26, 'gosh': 27, 'only': 28, 'could': 29, 'find': 30, 'kat': 31, 'a': 32, 'boyfriend': 33, 'ma': 34, 'this': 35, 'my': 36, 'head': 37, 'because': 38, 'it': 39, 'such': 40, 'nice': 41, 'one': 42, 'how': 43, 'our': 44, 'little': 45, 'date': 46, 'plan': 47, 'have': 48, 'word': 49, 'as': 50, 'gentleman': 51, 'do': 52, 'get': 53, 'hair': 54, 'to': 55, 'look': 56, 'like': 57, 'sure': 58, 'really': 59, 'wanna': 60, 'go': 61, 'but': 62, 'cannot': 63, 'unless': 64, 'sister': 65, 'goes': 66, 'kind': 67, 'of': 68, 'guy': 69, 'she': 70, 'likes': 71, 'pretty': 72, 'ones': 73, 'know': 74, 'looked': 75, 'for': 76, 'back': 77, 'at': 78, 'party': 79, 'always': 80, 'seemed': 81, 'be': 82, 'occupied': 83,

8096

In [0]:
for i in range(len(short_answers)):
    short_answers[i] += ' <EOS>'
    short_answers[i] = '<GO> ' + short_answers[i]

In [0]:
questions_int = []
for question in short_questions:
    ints = []
    for word in question.split():
        if word not in questions_vocab_to_int:
            ints.append(questions_vocab_to_int['<UNK>'])
        else:
            ints.append(questions_vocab_to_int[word])
    questions_int.append(ints)
    
answers_int = []
for answer in short_answers:
    ints = []
    for word in answer.split():
        if word not in answers_vocab_to_int:
            ints.append(answers_vocab_to_int['<UNK>'])
        else:
            ints.append(answers_vocab_to_int[word])
    answers_int.append(ints)

In [0]:
print(len(questions_int))
print(len(answers_int))

138335
138335


In [0]:
word_count = 0
unk_count = 0

for question in questions_int:
    for word in question:
        if word == questions_vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1
    
for answer in answers_int:
    for word in answer:
        if word == answers_vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1
    
unk_ratio = round(unk_count/word_count,4)*100
    
print("Total number of words:", word_count)
print("Number of times <UNK> is used:", unk_count)
print("Percent of words that are <UNK>: {}%".format(round(unk_ratio,3)))

Total number of words: 2472868
Number of times <UNK> is used: 92436
Percent of words that are <UNK>: 3.74%


In [0]:
sorted_questions = []
sorted_answers = []

for length in range(1, max_line_length+1):
    for i in enumerate(questions_int):
        if len(i[1]) == length:
            sorted_questions.append(questions_int[i[0]])
            sorted_answers.append(answers_int[i[0]])

print(len(sorted_questions))
print(len(sorted_answers))
print()
for i in range(3):
    print(sorted_questions[i])
    print(sorted_answers[i])
    print()

138335
138335

[58, 48]
[8096, 1, 59, 59, 59, 60, 61, 62, 1, 63, 12, 64, 36, 65, 66, 8094]

[1, 84]
[8096, 11, 192, 174, 55, 61, 21, 6, 8095, 159, 11, 8094]

[0, 85]
[8096, 153, 8, 9, 165, 11, 467, 55, 271, 8094]



In [0]:
def convert_to_sentence(sentences_int):
  sentences = []
  for sentence_int in sentences_int:
    sentence = ""
    for words_int in sentence_int:
      sentence += answers_int_to_vocab[words_int]
      sentence += " "
    sentences.append(sentence)
    
  return sentences

In [0]:
def padding_process(sentence_batch):
  lengths = [len(sentence) for sentence in sentence_batch]
  max_sentence = max([len(sentence) for sentence in sentence_batch])
  return [[sentence + [questions_vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch], lengths]

In [0]:
tf.reset_default_graph()
hm_epochs = 1
rnn_size = 256              #no of nodes in a layer
size_of_lexicon = 8097      #size of dict
embedding_size = 100        
max_gradient_norm = 5
learning_rate = 0.005
tgt_sos_id = 8094
tgt_eos_id = 8096
batch_size = 128
#num_layers = 2
#learning_rate_decay = 0.9
#min_learning_rate = 0.0001
#keep_probability = 0.75

x = tf.placeholder('int32', [None, None])
decoder_inputs = tf.placeholder('int32', [None, batch_size])
decoder_outputs = tf.placeholder('int32', [None, batch_size] )
input_sequence_length = tf.placeholder('int32', [None])
output_sequence_length = tf.placeholder('int32', [batch_size])
max_time = tf.placeholder('int32')
mode = tf.placeholder('int32')
tem_batch_size = tf.placeholder('int32')
embedding_encoder = tf.get_variable("embedding_encoder", [size_of_lexicon, embedding_size], dtype = tf.float32)
embedding_decoder = tf.get_variable("embedding_decoder", [size_of_lexicon, embedding_size], dtype = tf.float32)

decoder_cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size)
projection_layer = Dense(size_of_lexicon, use_bias=False)

In [0]:
def encoder_network(input_sentences, input_sequence_length):
  input_embeddings = tf.nn.embedding_lookup(embedding_encoder, input_sentences)
  encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size)
  #initial_state = encoder_cell.zero_state(batch_size, dtype=tf.float32)
  encoder_outputs, encoder_state = tf.nn.dynamic_rnn(encoder_cell, input_embeddings, sequence_length=input_sequence_length,
                                                     time_major=True, dtype=tf.float32)
  
  return encoder_state

In [0]:
def training_decoder_network(decoder_cell, decoder_input_sentences, encoder_state, output_sequence_length)  :
  
  print('in training_decoder_network')
  
  output_embeddings = tf.nn.embedding_lookup(embedding_decoder, decoder_input_sentences)
  
  helper = tf.contrib.seq2seq.TrainingHelper(output_embeddings, sequence_length = output_sequence_length, time_major=True)
  
  decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, helper, encoder_state, output_layer=projection_layer)
  
  final_outputs, final_state, final_sequence_lengths = tf.contrib.seq2seq.dynamic_decode(decoder, output_time_major= True)
  
  logits = final_outputs.rnn_output
  
  return logits

In [0]:
def testing_decoder_network(decoder_cell, encoder_state, input_sequence_length, tem_batch_size):
  
  print('in testing_decoder_network')
  
  helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embedding_decoder,tf.fill([tem_batch_size], tgt_sos_id), tgt_eos_id)
  
  decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, helper, encoder_state, output_layer=projection_layer)
  
  maximum_iterations = tf.round(tf.reduce_max(input_sequence_length[0]) * 5 )
  
  outputs, _, __ = tf.contrib.seq2seq.dynamic_decode(decoder, maximum_iterations=maximum_iterations)
  
  return outputs.sample_id

In [0]:
encoder_state = encoder_network(x, input_sequence_length)

logits = training_decoder_network(decoder_cell, decoder_inputs, encoder_state, output_sequence_length)

infer = testing_decoder_network(decoder_cell, encoder_state, input_sequence_length, tem_batch_size)  
crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=decoder_outputs, logits=logits)
target_weights = tf.sequence_mask(output_sequence_length, max_time, dtype=logits.dtype)
target_weights = tf.transpose(target_weights)
train_loss = (tf.reduce_sum(crossent * target_weights) /batch_size)
params = tf.trainable_variables()
gradients = tf.gradients(train_loss, params)
clipped_gradients, _ = tf.clip_by_global_norm(gradients, max_gradient_norm)

optimizer = tf.train.AdamOptimizer(learning_rate)
update_step = optimizer.apply_gradients(zip(clipped_gradients, params))

sess = tf.Session()


sess.run(tf.global_variables_initializer())

test_sentence = "how are you"

test = []
for word in test_sentence.split():
  test.append(questions_vocab_to_int[word])

test = np.array(test)
test = np.reshape(test, [1,3])
print(test)
test = np.transpose(test)
test_length = []
test_length.append(len(test))

for epoch in range(hm_epochs):
  epoch_loss = 0
  starting = 0
  i = 0
  while starting < 128*1:

    [padded_questions, questions_lengths] = padding_process(sorted_questions[starting : starting + batch_size])
    #print(i)
    i = i+1
    answers_with_eos = []
    answers_with_sos = []
    for answer in sorted_answers[starting : starting + batch_size]:
      answers_with_eos.append(answer[1:])
      answers_with_sos.append(answer[:-1])
    [padded_answers_with_eos, _] = padding_process(answers_with_eos)
    [padded_answers_with_sos, answers_lengths] = padding_process(answers_with_sos)

    padded_questions = np.array(padded_questions)
    padded_answers_with_eos = np.array(padded_answers_with_eos)
    padded_answers_with_sos = np.array(padded_answers_with_sos)


    padded_questions = np.transpose(padded_questions)
    padded_answers_with_eos = np.transpose(padded_answers_with_eos)
    padded_answers_with_sos = np.transpose(padded_answers_with_sos)

    starting = starting + batch_size
    _ = sess.run(encoder_state, feed_dict = {x: padded_questions, input_sequence_length : questions_lengths})
    
    print(np.shape(_))
    
    _, loss = sess.run([update_step, train_loss], feed_dict={x:padded_questions, decoder_inputs : padded_answers_with_sos,
                                                             decoder_outputs : padded_answers_with_eos, 
                                                             input_sequence_length : questions_lengths,
                                                             output_sequence_length : answers_lengths,
                                                             max_time : max(answers_lengths), tem_batch_size : 128})
    epoch_loss += loss

    _logits = sess.run(infer, feed_dict = {x: test, input_sequence_length : test_length, tem_batch_size : 1})
    #print(_logits)
    sentence = convert_to_sentence(_logits)
    print(sentence)
    
  print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
  

test_sentence = "how are you"
  
test = []
for word in test_sentence.split():
  test.append(questions_vocab_to_int[word])

test = np.array(test)
test = np.reshape(test, [1,3])
#print(test)
test = np.transpose(test)
length = []
length.append(len(test))

_logits = sess.run(infer, feed_dict = {x: test, input_sequence_length : length, tem_batch_size : 1})
#print(_logits)
sentence = convert_to_sentence(_logits)
print(sentence)


in training_decoder_network
in testing_decoder_network
[[43 18 11]]
(2, 128, 256)


ResourceExhaustedError: ignored

In [0]:
for epoch in range(hm_epochs):
  epoch_loss = 0
  starting = 0
  i = 0
  learning_rate = 0.003
  while starting < 128*1080:

    [padded_questions, questions_lengths] = padding_process(sorted_questions[starting : starting + batch_size])
    #print(i)
    i = i+1
    answers_with_eos = []
    answers_with_sos = []
    for answer in sorted_answers[starting : starting + batch_size]:
      answers_with_eos.append(answer[1:])
      answers_with_sos.append(answer[:-1])
    [padded_answers_with_eos, _] = padding_process(answers_with_eos)
    [padded_answers_with_sos, answers_lengths] = padding_process(answers_with_sos)

    padded_questions = np.array(padded_questions)
    padded_answers_with_eos = np.array(padded_answers_with_eos)
    padded_answers_with_sos = np.array(padded_answers_with_sos)


    padded_questions = np.transpose(padded_questions)
    padded_answers_with_eos = np.transpose(padded_answers_with_eos)
    padded_answers_with_sos = np.transpose(padded_answers_with_sos)

    starting = starting + batch_size
    #_ = sess.run([encoder_state, logits], feed_dict = {x: padded_questions, input_sequence_length : questions_lengths, 
                                                      #decoder_inputs : padded_answers_with_sos, output_sequence_length : answers_lengths})

    _, loss = sess.run([update_step, train_loss], feed_dict={x:padded_questions, decoder_inputs : padded_answers_with_sos,
                                                             decoder_outputs : padded_answers_with_eos, 
                                                             input_sequence_length : questions_lengths,
                                                             output_sequence_length : answers_lengths,
                                                             max_time : max(answers_lengths), tem_batch_size : 128})
    epoch_loss += loss
    '''
    _logits = sess.run(infer, feed_dict = {x: test, input_sequence_length : test_length, tem_batch_size : 1})
    #print(_logits)
    sentence = convert_to_sentence(_logits)
    print(sentence)
    '''
  print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

Epoch 0 completed out of 20 loss: 30146.52967453003
Epoch 1 completed out of 20 loss: 30033.689407348633
Epoch 2 completed out of 20 loss: 29931.156503677368
Epoch 3 completed out of 20 loss: 29850.385486602783
Epoch 4 completed out of 20 loss: 29769.464994430542
Epoch 5 completed out of 20 loss: 29697.08200454712
Epoch 6 completed out of 20 loss: 29651.688404083252
Epoch 7 completed out of 20 loss: 29581.09202003479
Epoch 8 completed out of 20 loss: 29538.6393699646
Epoch 9 completed out of 20 loss: 29469.207542419434
Epoch 10 completed out of 20 loss: 29451.046642303467
Epoch 11 completed out of 20 loss: 29396.30495071411
Epoch 12 completed out of 20 loss: 29366.549612045288
Epoch 13 completed out of 20 loss: 29316.22905921936
Epoch 14 completed out of 20 loss: 29282.707317352295
Epoch 15 completed out of 20 loss: 29260.479568481445
Epoch 16 completed out of 20 loss: 29238.658235549927
Epoch 17 completed out of 20 loss: 29233.424264907837
Epoch 18 completed out of 20 loss: 29209.1833

In [0]:
test_sentence = "i am ok"
  
test = []
for word in test_sentence.split():
  test.append(questions_vocab_to_int[word])

test = np.array(test)
test = np.reshape(test, [1,len(test_sentence.split())])
#print(test)
test = np.transpose(test)
length = []
length.append(len(test))

_logits = sess.run(infer, feed_dict = {x: test, input_sequence_length : length, tem_batch_size : 1})
#print(_logits)
sentence = convert_to_sentence(_logits)
print(sentence)

["sure you are <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> ' <EOS> <EOS> "]


In [0]:
variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
variables = sess.run(variables)
import pickle as pkl
pickle_file = open('pickle_file.pickle', 'wb')
pkl.dump(np.array(variables), pickle_file)
pickle_file.close()
print(len(variables))
print(np.shape(variables[0]))
print(np.shape(variables[1]))
print(np.shape(variables[2]))
print(np.shape(variables[3]))
print(np.shape(variables[4]))
print(np.shape(variables[5]))
print(np.shape(variables[6]))
'''
tf.reset_default_graph()
hm_epochs = 1
rnn_size = 256
size_of_lexicon = 8097
embedding_size = 100
max_gradient_norm = 5
learning_rate = 0.005
tgt_sos_id = 8094
tgt_eos_id = 8096
batch_size = 128
'''

7
(8097, 100)
(8097, 100)
(356, 1024)
(1024,)
(356, 1024)
(1024,)
(256, 8097)


'\ntf.reset_default_graph()\nhm_epochs = 1\nrnn_size = 256\nsize_of_lexicon = 8097\nembedding_size = 100\nmax_gradient_norm = 5\nlearning_rate = 0.005\ntgt_sos_id = 8094\ntgt_eos_id = 8096\nbatch_size = 128\n'

In [0]:
for epoch in range(hm_epochs):
  epoch_loss = 0
  starting = 0
  i = 0
  while starting < 128*1080:

    [padded_questions, questions_lengths] = padding_process(sorted_questions[starting : starting + batch_size])
    print(i)
    i = i+1
    answers_with_eos = []
    answers_with_sos = []
    for answer in sorted_answers[starting : starting + batch_size]:
      answers_with_eos.append(answer[1:])
      answers_with_sos.append(answer[:-1])
    [padded_answers_with_eos, _] = padding_process(answers_with_eos)
    [padded_answers_with_sos, answers_lengths] = padding_process(answers_with_sos)

    padded_questions = np.array(padded_questions)
    padded_answers_with_eos = np.array(padded_answers_with_eos)
    padded_answers_with_sos = np.array(padded_answers_with_sos)


    padded_questions = np.transpose(padded_questions)
    padded_answers_with_eos = np.transpose(padded_answers_with_eos)
    padded_answers_with_sos = np.transpose(padded_answers_with_sos)

    starting = starting + batch_size


    _, loss = sess.run([update_step, train_loss], feed_dict={x:padded_questions, decoder_inputs : padded_answers_with_sos,
                                                             decoder_outputs : padded_answers_with_eos, 
                                                             input_sequence_length : questions_lengths,
                                                             output_sequence_length : answers_lengths,
                                                             max_time : max(answers_lengths), tem_batch_size : 128})

    epoch_loss += loss

  print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649


745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994


1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
Epoch 0 completed out of 20 loss: 33814.566286087036
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237


363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612


...


621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870

943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
Epoch 18 completed out of 20 loss: 32802.94265937805
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
1

225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474


597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846


919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
Epoch 19 completed out of 20 loss: 32791.64496231079


In [0]:

with tf.Session() as sess:

  sess.run(tf.global_variables_initializer())
  starting = 0
  while starting < 128:
    [padded_questions, questions_lengths] = padding_process(sorted_questions[0 : 1])

    padded_questions = np.array(padded_questions)

    print(np.shape(padded_questions))

    print(questions_lengths)
    padded_questions = np.transpose(padded_questions)


    print(np.shape(padded_questions))
    _encoder_state = sess.run(encoder_state, feed_dict= {x:padded_questions, input_sequence_length:questions_lengths})
    print(_encoder_state)
    _infer = sess.run(infer, feed_dict={x:padded_questions, input_sequence_length : questions_lengths})

(1, 2)
[2]
(2, 1)
LSTMStateTuple(c=array([[-6.1660493e-04, -2.5790946e-03,  3.1082826e-03,  7.3614190e-03,
         3.9516343e-03, -5.9561832e-03, -6.9349525e-03, -3.5299649e-03,
        -2.7132309e-03, -5.2012089e-03,  1.1909783e-03,  8.7126653e-04,
         3.7732373e-03,  1.8491062e-03,  2.8082839e-04,  6.0467967e-03,
         3.8399501e-04, -2.4041370e-03,  3.1143669e-03,  7.7385632e-03,
         1.4187912e-03, -2.9321900e-04, -3.0533727e-03, -1.3794968e-02,
        -7.9846452e-04,  7.2837332e-03, -7.0795096e-03, -5.6209345e-03,
         1.1043495e-02,  8.9421467e-04, -2.9660042e-04, -3.3461703e-03,
        -8.1551084e-03, -3.9870203e-03,  3.6047495e-03,  1.3883202e-04,
        -3.9559621e-03,  4.1279197e-03,  4.4308398e-03, -3.0933400e-03,
         4.6970290e-03, -8.6953217e-04,  7.5592799e-03,  1.3589803e-03,
        -4.0214611e-03,  1.7749891e-03, -4.4412427e-03,  6.7510526e-04,
        -3.2509693e-03,  2.5072149e-03, -1.8210206e-03,  6.4897756e-03,
        -3.2364470e-03,  2.40

InvalidArgumentError: ignored